In [1]:
%load_ext autoreload
%autoreload 2


In [9]:
import yaml
import torch
from utils.tokenizer import load_tokenizer
from train import model_from_config

tok = load_tokenizer("tokenizer")

with open("../data/small_train.txt", "r") as f:
    f.readline() # the first line is just "text"
    data = f.read()
    
with open("../configs/llama_medium.yaml", "r") as f:
    config = yaml.safe_load(f)
    
model = model_from_config(config)
state_dict = torch.load("../46012.pt", map_location="cpu")["model_state_dict"]
state_dict = {k.replace("_orig_mod.", ""): v for k, v in state_dict.items()}  # Remove "_orig_mod." prefix from keys
model.load_state_dict(state_dict, strict=True)

/var/folders/5h/bc4n_w3d6pbcdc772w_c6tsc0000gn/T/ipykernel_17205/3859147415.py:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load("../46012.pt", map_l

<All keys matched successfully>

In [32]:
from utils.sample import sample_with_temp, nucleus_sample
import torch

def generate_text(model, tokenizer, prompt, max_length=50, temperature=0.7):
    model.eval()
    
    # Encode the prompt to input IDs
    input_ids = torch.tensor(tokenizer.encode(prompt, add_special_tokens=False).ids).unsqueeze(0)
    
    generated = []
    for _ in range(max_length):
        with torch.no_grad():
            outputs = model(input_ids)
            next_token_logits = outputs.logits[:, -1, :]
            # next_token = nucleus_sample(next_token_logits, 0.4)
            next_token = sample_with_temp(next_token_logits, temperature)
        
        generated.append(next_token.item())
        input_ids = torch.cat([input_ids, next_token.unsqueeze(0)], dim=-1)
        
        # Check for end-of-sequence token
        if next_token.item() == tokenizer.token_to_id("<eos>"):
            break
    

        token = tokenizer.id_to_token(next_token.item())

        print(token, end=" ")
        

# Example usage
prompt = "Once upon a time, Felix"
print(prompt, end=" ")
generate_text(model, tok, prompt, max_length=128)


Once upon a time, Felix was feeling very sleepy . He wanted to take a nap , but he was so sleepy that he couldn ' t . He decided to take a nap in the morning . When he woke up , he felt much better . He was so happy that he decided to take a nap right there in the morning . He was so happy that he had taken a nap in the morning . The end ." " Once upon a time , there was a girl named Sarah . She was three years old and loved to play . One day , Sarah was playing in the park when she saw a big puddle . She was so excited and wanted to jump in it . She 

In [69]:
with open("../data/small_train.txt", "r") as f:
    f.readline() # the first line is just "text"
    data = f.read()
    
# first 10000 words
data = " ".join(data.split()[:10000])
    
data = data.lower()
data = data.replace(".", "")
data = data.replace(",", "")
data = data.replace("!", "")
data = data.replace("?", "")
data = data.replace("'", "")
data = data.replace("\"", "")

word_to_ids = {word: i for i, word in enumerate(set(data.split()))}
ids_to_word = {i: word for word, i in word_to_ids.items()}

In [70]:
tokenized_data = [word_to_ids[word] for word in data.split()]

In [72]:
from utils import NGramTrie

trie = NGramTrie.fit(tokenized_data, ngram_max_length=3)

In [96]:
trie.search([word_to_ids[word] for word in "one day".split()]), data.count("one day")

(56, 56)

In [97]:
trie.search([word_to_ids[word] for word in "one day a".split()]), data.count("one day a")

(11, 11)

In [98]:
trie.search([word_to_ids[word] for word in "one day a".split()], "++*"), data.count("one day a")

(56, 11)

In [101]:
node = trie.find_all_nodes([word_to_ids[word] for word in "one day".split()])
[(ids_to_word[token], node.count) for token, node in node[0].children.items()]

[('a', 11),
 ('beep', 1),
 ('the', 7),
 ('while', 1),
 ('tim', 6),
 ('max', 1),
 ('when', 2),
 ('timmy', 1),
 ('his', 2),
 ('sue', 1),
 ('she', 8),
 ('he', 4),
 ('it', 1),
 ('zigzag', 1),
 ('they', 1),
 ('her', 1),
 ('tims', 1),
 ('joe', 1),
 ('mia', 1),
 ('lucy', 1),
 ('charlies', 1),
 ('sam', 1),
 ('bill', 1)]

In [106]:
data.count("one day while"), data.count("one day tim"), data.count("one day she")  # why is tim wrong?

(1, 8, 8)